In [14]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
from string import punctuation
punct = punctuation+'«»—…“”*№–'
import numpy as np
import math


# ПЛАН ДЕЙСВТИЙ


2) СХЛОПНУТЬ УНИГРАММЫ ИДЕНТИЧНЫЕ (за счет проставления айдишников верных лемм?) --- если нужно - заполняем хэшом а если не нужно то зануляем хэш

3) БРАТЬ ЗА ОСНОВУ МНОГЗНАЧНОСТЬ ИЗ ИСТОЧНИКОВ И К НЕЙ ПОДТИЯГИВАТЬ СУЩЕСТВУЮЩИЕ СЛУЧАИ

---


1) ДОБИТЬ АЛГОРИТМ ДИЗАМБИГУАЦИИ ЗА СЧЕТ ВОРД2ВЕК

In [8]:
def tokenize(text):
    #print(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stop_words]
    words = [word for word in words if word]
    words = [wordnet_lemmatizer.lemmatize(w) for w in words] 

    return words

In [16]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [30]:
model.similarity('crash', 'car')

0.616397

In [17]:
model['mother']

array([ 0.44615  , -0.3426   , -0.16236  ,  0.70221  , -0.0063556,
       -0.66198  ,  0.50239  ,  0.72514  ,  0.40139  , -0.19318  ,
       -0.55705  ,  0.47563  , -3.7614   ,  0.062386 , -0.28429  ,
        0.64277  ,  0.14114  , -0.17907  , -0.084768 ,  0.10419  ,
        0.29275  , -0.56099  , -0.58851  ,  0.30293  ,  0.91432  ,
        0.18194  ,  0.022049 , -0.26197  ,  0.17343  ,  0.13126  ,
        0.088969 , -0.66559  , -0.038818 ,  0.23208  , -0.55856  ,
        1.2076   ,  0.41975  ,  0.43664  , -0.18751  , -0.046753 ,
       -0.23344  ,  0.15437  ,  0.14866  ,  0.11116  ,  0.31672  ,
        0.23532  ,  0.73886  ,  0.32192  ,  0.27266  ,  0.26554  ,
       -0.41621  ,  0.0088457, -0.035569 , -0.51636  ,  0.27591  ,
       -0.26439  , -0.13055  ,  0.27277  , -0.16856  ,  0.48667  ,
       -0.21558  ,  0.33781  ,  0.6418   , -0.1669   , -0.1454   ,
        0.55402  , -0.19883  , -0.2383   ,  0.1369   ,  0.20284  ,
        0.096421 , -0.7681   , -0.1644   ,  0.012845 ,  0.5450

In [15]:
def lesk( word, sentence, debug = False, show_sentence = False,show_all = False):
    
    
    """Ваш код тут"""
    sentence =  sentence.split(" ")
    sentence = [w for w in sentence if not w in stop_words] 
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence = [wordnet_lemmatizer.lemmatize(w) for w in sentence]
    
    bestsense = 0
    #maxoverlap = 0
    min_mean_overlap_dist = 100#начальная точка новой метрики
    final_definition = ''
    
    word_index = sentence.index(word)#индекс исследуемого слова в предложении
    
    #получаем часть речи для дальнейшего отграничения ранжира поиска
    #pos_sentence = nltk.pos_tag(sentence)
    #word_pos = get_pos(pos_sentence, word)
    #if (debug): print("word_pos", word_pos)
    
    for i, synset in enumerate(wn.synsets(word)):
        #if (debug): print(i, synset)
        comparison_data = []    
        current_definition = synset.definition()
        
        definition = tokenize(current_definition)
        #print(definition)
        #if (debug):print("DEFINITION FULL",current_definition)
        
        #if (debug):print("DEFINITION PROCESSED",definition)
        
        
        examples =  synset.examples()
        comparison_data = [tokenize(ex_snetence) for ex_snetence in examples]
        sentence_set = set(sentence)
        #if (show_sentence): print("SETENCE",  sentence)
        
        #if (debug): print(definition & sentence_set)
        mean_overlap_distance = 0
        comparison_data.append(definition)#теперь сюда входят и примеры и само значение
        #print(comparison_data) 
        comparison_data_set = [set(el) for el in comparison_data]
        if (show_sentence): print("SETENCE",  sentence)
        if (debug): print("START ITERATING OVER AL DATA")
        
        word_distance = []
        for comp_sentence_set,comp_sentence in zip(comparison_data_set, comparison_data):
            if (debug):print("comp_sentence:",' '.join(comp_sentence))
            overlap_words_set = sentence_set & comp_sentence_set
            if(word in overlap_words_set):
                overlap_words_set.remove(word)

            if (overlap_words_set and debug):print("overlap_words_set",overlap_words_set)
            for overlapped_word in overlap_words_set:
                if (debug): print("disambig_word_index = ", word_index)
                ind = sentence.index(overlapped_word)
                if (debug):print("overlapped_word = ", overlapped_word)
                if (debug): print("overlapped_word_ind", ind)
                abs_distance = abs(ind - word_index)
                if (debug): print("abs_distance", abs_distance)
                word_distance.append(abs_distance)  
        
        
        #overlap_len = len(overlap)
        if (debug): print("word_distance" , word_distance)
        #применяем новую метрику
        mean_overlap_distance = np.mean(word_distance) / len(word_distance)
        if (debug):
            if ( mean_overlap_distance == 0 or math.isnan(mean_overlap_distance) ): 
                print("ITERATING FINISHED.NO OVERLAP FOUND",'\n')
        if(mean_overlap_distance > 0):
            if (debug):print("OVERLAP_FOUND")
            if (debug):print("EXAMPLES_ORIGINAL = ", examples)
            if (debug):print("ALL_COMPAR_DATA = ", comparison_data)
            if (debug): print(i, synset)
            if (debug):print("DEFINITION FULL",current_definition)
            if (debug):print("DEFINITION PROCESSED",definition)
            if (show_sentence): print("SETENCE",  sentence)
            if (debug): print("mean_overlap_distance", mean_overlap_distance)
                
            if (debug): print("\n")
        
        #обновляем посчитанное значение если оно меньше предыдущего
        if (mean_overlap_distance < min_mean_overlap_dist):
            min_mean_overlap_dist = mean_overlap_distance
            bestsense = i
            final_definition = current_definition
    
    return bestsense, final_definition
lesk("break", "They break the rules in order to convince the rule-makers",debug = True, show_sentence = True)#True False

SETENCE ['They', 'break', 'rule', 'order', 'convince', 'rule-makers']
START ITERATING OVER AL DATA
comp_sentence: telephone annoying interruption
comp_sentence: break action player hurt
comp_sentence: abrupt occurrence interrupt ongoing activity
word_distance []
ITERATING FINISHED.NO OVERLAP FOUND 

SETENCE ['They', 'break', 'rule', 'order', 'convince', 'rule-makers']
START ITERATING OVER AL DATA
comp_sentence: finally got big break
comp_sentence: unexpected piece good luck
word_distance []
ITERATING FINISHED.NO OVERLAP FOUND 

SETENCE ['They', 'break', 'rule', 'order', 'convince', 'rule-makers']
START ITERATING OVER AL DATA
comp_sentence: built right geological fault
comp_sentence: studied faulting earth's crust
comp_sentence: geology crack earth's crust resulting displacement one side respect
word_distance []
ITERATING FINISHED.NO OVERLAP FOUND 

SETENCE ['They', 'break', 'rule', 'order', 'convince', 'rule-makers']
START ITERATING OVER AL DATA
comp_sentence: hoped avoid break relatio

(28, 'fail to agree with; be in violation of; as of rules or patterns')